## Google Connection

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


## Installs 

In [2]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.2 MB/s 
     |████████████████████████████████| 4.7 MB 59.8 MB/s 
     |████████████████████████████████| 1.3 MB 75.3 MB/s 
     |████████████████████████████████| 120 kB 88.0 MB/s 
     |████████████████████████████████| 6.6 MB 57.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8abf2f9f8a999ee40032f66f50eba1b83bc95ec5b749278bb48b5d109bc59c5a
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [3]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 7.3 MB/s 
     |████████████████████████████████| 41 kB 712 kB/s 


In [4]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 9.9 MB/s 
     |████████████████████████████████| 3.5 MB 7.0 MB/s 


In [5]:
! pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 7.6 MB/s 


In [6]:
!pip install wandb -q

     |████████████████████████████████| 1.8 MB 8.6 MB/s 
     |████████████████████████████████| 158 kB 78.8 MB/s 
     |████████████████████████████████| 181 kB 88.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 157 kB 84.5 MB/s 
     |████████████████████████████████| 157 kB 89.3 MB/s 
     |████████████████████████████████| 157 kB 100.8 MB/s 
     |████████████████████████████████| 157 kB 98.6 MB/s 
     |████████████████████████████████| 157 kB 11.2 MB/s 
     |████████████████████████████████| 157 kB 82.6 MB/s 
     |████████████████████████████████| 157 kB 79.6 MB/s 
     |████████████████████████████████| 156 kB 95.9 MB/s 


## SetUp

In [7]:
from sentence_transformers import SentenceTransformer, util
import torch.nn.functional as F
import os

import logging
import os.path as osp
import ssl
import urllib.request
from rdflib import Graph, Literal, URIRef
import random
import numpy as np
from collections import Counter

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv

import xml.etree.ElementTree as etree  # from lxml import etree #for parser
from io import BytesIO
import logging
import tempfile
from six.moves import urllib
from xml.sax.saxutils import quoteattr, escape

from torchmetrics.functional import pairwise_cosine_similarity

In [8]:
import sys
main_files_path = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/MAGNET"

sys.path.append(os.path.abspath(main_files_path))

from load_data import download_url, create_pyg_data, create_fake_data
from stuff_for_model import calc_acc
from interpretation import count_occurences, check_real_val_of_both, check_single_left_vals, check_single_right_vals, save_output
from AlignmentFormat import *

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [9]:
from AlignmentFormat import *

In [10]:
import wandb

In [11]:
#bb237cce5fb5a8ae6844b07c8d9ae3b33598575b

In [12]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [13]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'magnet'

In [15]:
wandb.init(project="magnet")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Data

In [16]:
file_path = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/prep_data/"

In [17]:
data = torch.load(file_path+"anatomy_ressource_focused.pt")

In [18]:
data

Data(
  x_one=[3105, 384],
  edge_index_one=[2, 5957],
  edge_type_one=[5957],
  edge_attr_emb_one=[5957, 384],
  map_one=[5957, 384],
  res_map_one={
    http://mouse.owl#MA_0000861=0,
    http://mouse.owl#MA_0000426=1,
    http://mouse.owl#MA_0002115=2,
    http://mouse.owl#MA_0002582=3,
    http://mouse.owl#genid185=4,
    http://mouse.owl#MA_0000725=5,
    http://mouse.owl#MA_0002023=6,
    http://mouse.owl#MA_0002671=7,
    http://mouse.owl#MA_0002730=8,
    http://mouse.owl#MA_0000301=9,
    http://mouse.owl#MA_0000051=10,
    http://mouse.owl#MA_0000634=11,
    http://mouse.owl#MA_0000080=12,
    http://mouse.owl#MA_0002040=13,
    http://mouse.owl#MA_0000736=14,
    http://mouse.owl#genid1799=15,
    http://mouse.owl#MA_0001226=16,
    http://mouse.owl#MA_0002212=17,
    http://mouse.owl#MA_0002474=18,
    http://mouse.owl#MA_0001153=19,
    http://mouse.owl#MA_0002086=20,
    http://mouse.owl#genid125=21,
    http://mouse.owl#MA_0001407=22,
    http://mouse.owl#MA_0002098=23,


In [19]:
double_left_tensor, double_right_tensor = create_fake_data(data.train_set_right, data.train_set_left)

## Base Model

Anleitung zum Pytorch Model schreiben: https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

**To be built in:**

Ideas from SBert:

- Mean Pooling (die probieren auch andere Poolings aus aber bleiben dann bei dem) -> global_mean_pool in geometric oder die haben auch selbst was geschrieben: https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/models/Pooling.py
- Batch size (used a batch-size of 16)
- Optimizer (, Adam optimizer with learning rate 2e−5, and a linear learning rate warm-up over 10% of the training data). 

In [20]:
class MyModel(torch.nn.Module):
    def __init__(self, num_node_features):
        super().__init__()
        self.conv1_one = GCNConv(num_node_features, num_node_features)
        self.conv1_two = GCNConv(num_node_features, num_node_features)

    def forward(self, data):
        
        x_one, edge_index_one = data['x_one'], data['edge_index_one']
        x_one = self.conv1_one(x_one, edge_index_one).relu() 
        x_one = F.dropout(x_one, training=self.training)
        
        x_two, edge_index_two = data['x_two'], data['edge_index_two']
        x_two = self.conv1_two(x_two, edge_index_two).relu()
        x_two = F.dropout(x_two, training=self.training)
        return x_one, x_two


class CosineSimilarityLoss(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.loss_fct = torch.nn.MSELoss()

    def forward(self, data):
        x_one, x_two = self.model(data)
        
        # hier werden die "wichtigen" Embeddings gefiltert - von den Entitäten, die später Paare bilden
        left_embeddings_double = torch.index_select(x_one, 0, double_left_tensor)
        right_embeddings_double = torch.index_select(x_two, 0, double_right_tensor)

        # hier wird die similarity von den embeddings gemessen - je höher, desto besser, weil das ja jeweils genau die sehr ähnlichen Embeddings sein sollten
        output = torch.cosine_similarity(left_embeddings_double, right_embeddings_double)

        # gibt den mean squared error loss aus - je kleiner, desto besser 
        return self.loss_fct(output, torch.cat(((torch.ones(int(len(left_embeddings_double)/2)), torch.zeros(int(len(left_embeddings_double)/2)))))) # negatives

In [21]:
device = 'cpu' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')
source_data = data.to(device)
model = MyModel(384).to(device)
myloss = CosineSimilarityLoss(model)
double_left_tensor = double_left_tensor.to(device)
double_right_tensor = double_right_tensor.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.0005, momentum=0.9)
double_left_tensor.to(device)
double_right_tensor.to(device)

wandb.config = {
  "learning_rate": 0.1,
  "weight_decay":0.0005,
  "momentum":0.9,
  "epochs": 1000,
  "optimizer": "SGD",
}

def train():
    model.train()
    optimizer.zero_grad()
    loss = myloss(source_data)
    loss.backward()
    optimizer.step()
    return float(loss)

In [22]:
def calc_acc(lefts, rights):
    output = util.semantic_search(lefts, rights, top_k=1)
    matches = []
    matches_count = 0
    for i in range(0, len(output)):
      matches.append([i, output[i][0]['corpus_id']])
      if i == output[i][0]['corpus_id']:
        matches_count = matches_count + 1
    return matches_count / len (output), matches

In [23]:
@torch.no_grad()
def test_easier():
    model.eval()
    pred = model(source_data)
    # take subset to minimize searching time -> only now, normally we should do it better

    lefties_train = pred[0][source_data.train_set_left]
    righties_train = pred[1][source_data.train_set_right]
    lefties_test = pred[0][source_data.test_set_left]
    righties_test = pred[1][source_data.test_set_right]

    # Trying out the function of SentenceBert
    train_acc, matches_train = calc_acc(lefties_train, righties_train)
    test_acc, matches_test = calc_acc(lefties_test, righties_test)

    return train_acc, test_acc, matches_train, matches_test

In [24]:
def calc_acc_hard(indexed_set, lefties, righties):  
  matches_count = 0
  wrong_matches_count = 0
  all_matches_entities = []

  output = util.semantic_search(lefties, righties, top_k=1)

  for i, out in zip(range(0, len(output)), output):
    if (out[0]['score'] > 0.9):
      try:
        if indexed_set[i] == out[0]['corpus_id']:
          matches_count = matches_count + 1
          all_matches_entities.append([i, output[i][0]['corpus_id']])
        else:
          wrong_matches_count = wrong_matches_count + 1
          all_matches_entities.append([i, output[i][0]['corpus_id']])
      except:
        wrong_matches_count = wrong_matches_count + 1
        all_matches_entities.append([i, output[i][0]['corpus_id']])
    else:
      pass
  try:
    #acc = matches_count / (matches_count + wrong_matches_count)
    acc = matches_count / len(indexed_set)
  except: 
    acc = 0
  return acc, all_matches_entities

In [25]:
def calc_prec_rec(lefties, righties, gold_standard):
  matches_all = []
  matches_all_complete = []
  match_count = 0
  false_positives = 0
  output = util.semantic_search(lefties, righties, top_k=1)
  for i, out in zip(range(0, len(output)), output):
    if (out[0]['score'] > 0.9):
      matches_all_complete.append([i, out[0]['corpus_id'], out[0]['score']])
      matches_all.append([i, out[0]['corpus_id']])
  for match in matches_all:
    if match in gold_standard:
      match_count = match_count +1
    else:
      false_positives = false_positives + 1
  false_negatives = len(gold_standard) - match_count
  try:
    prec =  match_count / (match_count + false_positives) # wenn das gering ist, wählt er zu viele aus
  except:
    prec = 0 
  recall = match_count / (match_count + false_negatives) # wenn das gering ist, wählt er nicht genügend aus?, wenn hoch, dann wählt er viele von der tatsächlichen Menge aus
  return prec, recall, matches_all_complete

In [26]:
@torch.no_grad()
def test_harder():
    model.eval()
    pred = model(source_data)
    #train_set = dict(zip(data.train_set_left.tolist(), data.train_set_right.tolist()))
    #test_set = dict(zip(data.test_set_left.tolist(), data.test_set_right.tolist()))
    gold_standard = list(map(list, zip(data.left_indices.tolist(), data.right_indices.tolist())))

    lefties = pred[0]
    righties = pred[1]

    #train_acc, matches_train = calc_acc_hard(train_set, lefties, righties)
    #test_acc, matches_test = calc_acc_hard(test_set, lefties, righties)
    prec, rec, matches = calc_prec_rec(lefties, righties, gold_standard)

    wandb.log({
        "Matches": matches,
        "Precision": prec,
        "Recall": rec})

    return prec, rec, matches

In [ ]:
wandb.watch(model, log="all")

In [27]:
for epoch in range(1, 1000):
    loss = train()
    #train_acc_easy, test_acc_easy, matches_train_easy, matches_test_easy = test_easier()
    #print(f'EASY \n Epoch: {epoch:02d}, Loss: {loss:.4f}, Train: {train_acc_easy:.4f} , Test: {test_acc_easy:.4f} ')
    #train_acc_hard, test_acc_hard, matches_train_hard, matches_test_hard = test_harder()
    #print(f'HARD \n Epoch: {epoch:02d}, Loss: {loss:.4f}, Train: {train_acc_hard:.4f} , Test: {test_acc_hard:.4f} ')
    prec, rec, matches = test_harder()
    print(f'HARD \n Epoch: {epoch:02d}, Loss: {loss:.4f}, Precision: {prec:.4f} , Recall: {rec:.4f} ')

HARD 
 Epoch: 01, Loss: 0.3732, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 02, Loss: 0.3703, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 03, Loss: 0.3637, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 04, Loss: 0.3556, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 05, Loss: 0.3475, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 06, Loss: 0.3366, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 07, Loss: 0.3272, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 08, Loss: 0.3166, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 09, Loss: 0.3068, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 10, Loss: 0.2973, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 11, Loss: 0.2898, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 12, Loss: 0.2864, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 13, Loss: 0.2805, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 14, Loss: 0.2755, Precision: 0.0000 , Recall: 0.0000 
HARD 
 Epoch: 15, Loss: 0.2745, Precision: 0.000

# Save outputs

In [ ]:
def save_output(matches_test, data):
    resource_position_map_one_rev = {y: x for x, y in data.res_map_one.items()}
    resource_position_map_two_rev = {y: x for x, y in data.res_map_two.items()}
    uris_left = []
    #for ind in matches_test:
        #uris_left.append(check_single_left_vals(data, resource_position_map_one_rev, ind[0]))
    uris_right = []
    #for ind in matches_test:
     #   uris_right.append(check_single_right_vals(data, resource_position_map_two_rev, ind[1]))
    for i in range(0, len(matches)):
      uris_left.append(resource_position_map_one_rev[matches[i][0]])
      uris_right.append(resource_position_map_two_rev[matches[i][1]])
    conf_scores = [el[2] for el in matches_test]
    relation = ['='] * len(conf_scores)
    output_test = list(zip(uris_left, uris_right, relation, conf_scores))
    return output_test


In [ ]:
output = save_output(matches, source_data)

In [ ]:
result_path = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/results"

In [ ]:
name = model_type + '_' + optimizer_name + '_'

NameError: ignored

In [ ]:
serialize_mapping_to_file(result_path+'/test1.txt', output)

# Save model

In [ ]:
PATH = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/models/model_1"

In [ ]:
torch.save(model.state_dict(), PATH)

# Load model

In [ ]:
PATH = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/model_1/models"
loaded_model = MyModel(384)
loaded_model.load_state_dict(torch.load(PATH))
loaded_model.eval()
pred = loaded_model(source_data)

lefties_val = pred[0][source_data.val_set_left]
righties_val = pred[1][source_data.val_set_right]

val_acc, matches_val = calc_acc(lefties_val, righties_val)

val_acc

# Interpretation

In [ ]:
occurences = count_occurences(matches_train_easy, matches_test_easy, 10)

In [ ]:
print(f"In the train set, {occurences[0][0][0]} was the most common match ({occurences[0][0][1]} times) ")
print(f"In the test set, {occurences[1][0][0]} was the most common match ({occurences[1][0][1]} times) ")

In the train set, 498 was the most common match (356 times) 
In the test set, 324 was the most common match (99 times) 


In [ ]:
resource_position_map_one_rev = {y: x for x, y in resource_position_map_one.items()}
resource_position_map_two_rev = {y: x for x, y in resource_position_map_two.items()}

In [ ]:
resource_position_map_two_rev = dict((element, index) for (index, element) in enumerate(resource_position_map_two))

In [ ]:
print(f"A lot of entities in the train set were mistaken for this entity: {check_real_val_of_both(source_data,  resource_position_map_one_rev, resource_position_map_two_rev, occurences[0][0][0])}")
print(f"A lot of entities in the test set were mistaken for this entity: {check_real_val_of_both(source_data,  resource_position_map_one_rev, resource_position_map_two_rev, occurences[1][0][0])}")

A lot of entities in the train set were mistaken for this entity: ('http://mouse.owl#MA_0001797', 'http://human.owl#NCI_C13075')
A lot of entities in the test set were mistaken for this entity: ('http://mouse.owl#MA_0002442', 'http://human.owl#NCI_C13058')


# Back Up Code

Der Code ist für nicht-gefakte Embeddings gedacht

In [ ]:

#left_embeddings = torch.index_select(x_one, 0, data.train_set_left)
#right_embeddings = torch.index_select(x_two, 0, data.train_set_right)

# hier wird die similarity von den embeddings gemessen - je höher, desto besser, weil das ja jeweils genau die sehr ähnlichen Embeddings sein sollten
#output = torch.cosine_similarity(left_embeddings, right_embeddings)

# gibt den mean squared error loss aus - je kleiner, desto besser 
#return self.loss_fct(output, torch.cat(((torch.ones(int(len(left_embeddings_double)/2)), torch.zeros(int(len(left_embeddings_double)/2)))))) # negatives
#return self.loss_fct(output, torch.ones(int(len(data.train_set_left))))


In [ ]:
@torch.no_grad()
def test_harder():
    model.eval()
    pred = model(source_data)

    # take subset to minimize searching time -> only now, normally we should do it better
    lefties_train = pred[0][source_data.train_set_left]
    righties_train = pred[1][source_data.train_set_right]
    lefties_test = pred[0][source_data.test_set_left]
    righties_test = pred[1][source_data.test_set_right]

    all_lefties = pred[0]
    all_righties = pred[1]

    # Trying out the function of SentenceBert
    train_acc, matches_train = calc_acc(lefties_train, righties_train)
    test_acc, matches_test = calc_acc(lefties_test, righties_test)

    return train_acc, test_acc, matches_train, matches_test

# Ausprobierzone

In [ ]:
import re

Wir gehen davon aus, dass die bezeichnende Ressource immer am Ende von einer URL steht -> wollen, dass es ein echtes Wort ist :)=

In [ ]:
s = 'http://mouse.owl#MA_0002110'

In [ ]:
s2 = 'http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Category:Tatooine_quests'

In [ ]:
re.split(r'\#', s)

['http://mouse.owl', 'MA_0002110']

In [ ]:
re.split(r'\/', s)

['http:', '', 'mouse.owl#MA_0002110']

In [ ]:
re.split(r'\/', s2)

['http:',
 '',
 'dbkwik.webdatacommons.org',
 'swg.wikia.com',
 'resource',
 'Category:Tatooine_quests']

In [ ]:
re.split(r'\#', s2)

['http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Category:Tatooine_quests']

In [ ]:
# wir haben einen eingehenden String (muss eine ressource sein)


if len(re.split(r'\#', s)) > 1:
  print('hallo')
elif len(re.split(r'\/', s)) > 1:
  print('hi')

hallo


In [ ]:
s.split('/')

['http:', '', 'mouse.owl#MA_0002110']

# Batch Creation

In [ ]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

class PairData(Data):
    def __init__(self, edge_index_s=None, x_s=None, edge_index_t=None, x_t=None):
        super().__init__()
        self.edge_index_s = edge_index_s
        self.x_s = x_s
        self.edge_index_t = edge_index_t
        self.x_t = x_t

    def __inc__(self, key, value, *args, **kwargs):
      if key == 'edge_index_s':
          return self.x_s.size(0)
      if key == 'edge_index_t':
          return self.x_t.size(0)
      else:
          return super().__inc__(key, value, *args, **kwargs)

pair_data = PairData(data.edge_index_one, data.x_one, data.edge_index_two, data.x_two)
data_list = [pair_data, pair_data]

loader = DataLoader(data_list, batch_size=10, follow_batch=['x_s', 'x_t'])

# Batch Pipeline

In [ ]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(loader):

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(data)

        # Compute the loss and its gradients
        loss = myloss(data)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(loader) + i + 1
            running_loss = 0.

    return last_loss